# How do I make a new _project_?
### Overview
There are a number of API calls related to projects. Here we focus on _creating a **new**_ project. Along the way, we will also show how to [list billing groups](http://docs.cancergenomicscloud.org/docs/list-your-billing-groups).

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly here).
  
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify credentials file profile {cgc, sbg, default}
prof = 'default'

config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Make a new project using your first billing group
We start by listing all of your projects and your billing groups. Next we create the JSON that will be passed to the API to create the project. The dictionary should include:
* **billing_group** *Billing group* that will be charged for this project
* **description**   (optional) Project description
* **name**   Name of the project, may be *non-unique*<sup>1</sup>

**After** creating the project, you can re-check the project list. A **detail**-call for projects returns the following *attributes*:
* **description** The user specified project description
* **id**     _Unique_ identifier for the project, generated based on Project Name
* **name**   Name of project specified by the user, maybe _non-unique_
* **href**   Address<sup>1</sup> of the project.
* **tags**   List of tags
* **created_on** Project creation time
* **modified_on** Project modification time
* **created_by** User that created the project
* **root_folder** ID of the root folder for that project
* **billing_group** ID of the billing group for the project
* **settings** Dictionary with project settings for storage and task execution


<sup>1</sup> Please **don't** use non-unique *project names*. However, if you insist (on the GUI), the backend will allow it and assign a unique **id** to you project.

In [ ]:
# [USER INPUT] Set project name and billing group index here:
new_project_name = 'Michael Diamond'                         
index_billing = -1   

# Check if this project already exists. LIST all projects and check for name match
my_project = api.projects.query(name=new_project_name)
              
if my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('A project named {} exists, please choose a unique name'
          .format(new_project_name))
    raise KeyboardInterrupt
else:
    # Create a new project
    # What are my funding sources?
    billing_groups = api.billing_groups.query()  
    print((billing_groups[index_billing].name + \
           ' will be charged for computation and storage (if applicable)'))

    # Set up the information for your new project
    new_project = {
            'billing_group': billing_groups[index_billing].id,
            'description': """A project created by the API recipe (projects_makeNew.ipynb).
                          This also supports **markdown**
                          _Pretty cool_, right?
                       """,
            'name': new_project_name
    }

    my_project = api.projects.create(
        name=new_project['name'], billing_group=new_project['billing_group'], 
        description=new_project['description']
    )
    
    # (re)list all projects, and get your new project
    my_project = [p for p in api.projects.query(limit=100).all() 
              if p.name == new_project_name][0]

    print('Your new project {} has been created.'.format(
        my_project.name))
    # Print description if it exists
    if hasattr(my_project, 'description'): 
        print('Project description: \n {}'.format(my_project.description)) 

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-new-project)